In [1]:
import pandas as pd

# Arguments
message = '<image>\nThis is demo only. not real. do your best. These frames are captured for a potential traffic incident. Give me quanitative information whenever possible. Give me the following and number each answer:\nNumber of vehicles in accident in a number,\nAccident Type such as t-bone, rear end, etc,\nPerson Injury yes or no,\nNeed for ambulance yes or no,\nNeed for firetruck yes or no,\nNeed for Police yes or no,\nTypes of vehicles involved, such as suv, truck, sedan,\nFire yes or no,\nDay/night and weather, such as clear, etc,\nLow Res/Bad Footage yes or no.\nPlease ignore any context before these images and this prompt'
path = 'combined_images/'

#the message literally has \n in it, so we need to split by \n and then join by \n

message = message.replace("\n", "\\n")

df = pd.read_csv('accident_ground_truths.csv', index_col="Unnamed: 0")
df = df.fillna("N/A")
#df = df.drop(columns=["Other"])
#Print all the column names

print(df.columns)

Index(['Number of vehicles in accident', 'Accident Type', 'Person Injury? ',
       'Need for ambulance?', 'Need for firetruck?', 'Need for Police?',
       'Types of vehicles involved', 'Fire?', 'Weather',
       'Low Res/Bad Footage?', 'Other'],
      dtype='object')


In [2]:
#Start writing the JSON file

def grab_gpt_message(row) -> str:
    return "Number of vehicles in accident: " + str(row["Number of vehicles in accident"]) + "\\n" + "Accident Type: " + str(row["Accident Type"]) + "\\n" + "Person Injury: " + str(row["Person Injury? "]) + "\\n" + "Need for ambulance: " + str(row["Need for ambulance?"]) + "\\n" + "Need for firetruck: " + str(row["Need for firetruck?"]) + "\\n" + "Need for Police: " + str(row["Need for Police?"]) + "\\n" + "Types of vehicles involved: " + str(row["Types of vehicles involved"]) + "\\n" + "Fire: " + str(row["Fire?"]) + "\\n" + "Time of Day and weather: " + str(row["Weather"]) + "\\n" + "Low Res/Bad Footage: " + str(row["Low Res/Bad Footage?"]) + "\\n" + "Other Information: " + str(row["Other"])


with open('accident_ground_truths.json', 'w') as f:
    f.write('[\n')
    for index, row in df.iterrows():
        f.write('\t{\n')
        f.write(f'\t\t"id": "{index}",\n')
        f.write(f'\t\t"path": "{path}{index:06d}.jpg",\n')
        f.write(f'\t\t"conversations": [\n')
        f.write('\t\t\t{\n')
        f.write(f'\t\t\t\t"from": "human",\n\t\t\t\t"value": "{message}"\n')
        f.write('\t\t\t},\n')
        f.write('\t\t\t{\n')
        f.write(f'\t\t\t\t"from": "gpt",\n\t\t\t\t"value": "{grab_gpt_message(row)}"\n')
        f.write('\t\t\t}\n')
        f.write('\t\t]\n')
        f.write('\t}')
        if index != len(df) - 1:
            f.write(',')
        f.write('\n')
    f.write(']')